In [1]:
cd "../"

C:\Users\msi\Desktop\Research lab\Covid_AI_Research_Lab\Machine_Learning


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys
from os.path import abspath

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.get_logger().setLevel('ERROR')

import tensorflow.keras.backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from art.estimators.classification import KerasClassifier
from art.attacks.poisoning import PoisoningAttackBackdoor, PoisoningAttackCleanLabelBackdoor
from art.attacks.poisoning.perturbations import add_pattern_bd
from art.utils import load_mnist, preprocess, to_categorical
from art.defences.trainer import AdversarialTrainerMadryPGD
from tensorflow import keras
from tensorflow.keras import layers


In [4]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 2CEC-A46B

 Directory of C:\Users\msi\Desktop\Research lab\Covid_AI_Research_Lab\Machine_Learning\Attacking using ART\LowProFool

10/19/2023  04:42 PM    <DIR>          .
10/19/2023  04:42 PM    <DIR>          ..
10/19/2023  04:42 PM    <DIR>          .ipynb_checkpoints
10/19/2023  04:42 PM            18,060 Low Pro Fool.ipynb
10/10/2023  02:15 AM           267,328 low_pro_fool_adversaries.npy
               2 File(s)        285,388 bytes
               3 Dir(s)   4,316,143,616 bytes free


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load dataset
data = pd.read_excel(r"C:\Users\msi\Desktop\Research lab\Covid_AI_Research_Lab\Machine_Learning\merged_scored_dataset1.xlsx")

# Convert the target variable based on the given conditions
data['flag'] = 1
# data.loc[data['Laboratory confirmed, since the beginning of the pandemic Hospitalized'] <= 449, 'flag'] = 1
data.loc[data['Laboratory confirmed, since the beginning of the pandemic Hospitalized'] <= 290, 'flag'] = 0

# Drop the columns that exist in the dataset and ignore any errors for columns not found
columns_to_drop = ['District', 'Recovery Rate', 'Cluster','Hospitalization Rate']
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns])

# Split the data into training and test sets
X = data.drop(columns=['Laboratory confirmed, since the beginning of the pandemic Hospitalized', 'flag'])
y = data['flag']

train_data_flag, test_data_flag, y_train_flag, y_test_flag = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
mean_flag = train_data_flag.mean(axis=0)
std_flag = train_data_flag.std(axis=0)
train_data_flag = (train_data_flag - mean_flag) / std_flag
test_data_flag = (test_data_flag - mean_flag) / std_flag


In [5]:
x_train, x_test, y_train, y_test = train_data_flag, test_data_flag, y_train_flag, y_test_flag 
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [6]:
train_data_flag.shape, test_data_flag.shape, y_train_flag.shape, y_test_flag.shape

((13360, 10), (3340, 10), (13360,), (3340,))

In [7]:
# Normalize the data
mean_flag = train_data_flag.mean(axis=0)
std_flag = train_data_flag.std(axis=0)

In [7]:
# from tensorflow import keras
# from tensorflow.keras import layers
# # tf.compat.v1.enable_eager_execution()


# # Model Building
# model = keras.Sequential([
#     layers.Dense(128, activation='relu', input_shape=(train_data_flag.shape[1],)),
#     layers.Dropout(0.2), #to prevent overfitting
#     layers.Dense(64, activation='relu'),
#     layers.Dense(4, activation='softmax')  # We have 3 classes (1, 2, 3), but due to indexing, it's better to set this to 4 and ignore the 0th index
# ])

# model.compile(optimizer='adam', 
#                    loss='sparse_categorical_crossentropy', 
#                    metrics=['accuracy'] )
# #model training
# history_flag = model.fit(train_data_flag, y_train_flag, epochs=30, batch_size=64, validation_split=0.2)


# # Display training history for review
# history_flag.history


In [8]:
# Import SVC
from sklearn.svm import SVC

# import ART wrapper for scikit-learn SVC
from art.estimators.classification.scikitlearn import ScikitlearnSVC

In [9]:
# Initialize and fit SVC
svc_clf = SVC()
svc_clf.fit(x_train.values, y_train) 

SVC()

In [10]:
from art.estimators.classification.scikitlearn import ScikitlearnSVC
scaled_clip_values_cancer = (-1., 1.)

# Wrap SVC using the wrapper
classif_svc = ScikitlearnSVC(
    model = svc_clf,
    clip_values = scaled_clip_values_cancer
)

In [11]:
x_train.values

array([[ 1.7371415 ,  0.41283384,  0.86754864, ..., -0.16270088,
         0.68501157,  1.5905991 ],
       [ 1.0775176 , -0.35493907, -0.2221096 , ..., -0.5847207 ,
        -0.15169121, -0.606425  ],
       [-0.91573083, -0.6970161 , -0.7485737 , ..., -0.703002  ,
        -0.86948955,  0.12858231],
       ...,
       [-0.24974892, -0.26371852,  2.7040515 , ..., -0.47237507,
        -0.7530331 ,  1.3957795 ],
       [-0.9638907 , -0.71221954, -0.69960034, ..., -0.64132774,
        -0.82927185,  1.3325151 ],
       [-0.3696029 , -0.67421097, -0.36902982, ..., -0.7354317 ,
        -0.606326  , -2.5562208 ]], dtype=float32)

In [12]:
from art.attacks.evasion import LowProFool

lpf_svc = LowProFool(
    classifier = classif_svc,
    n_steps    = 15,
    eta        = 15,
    lambd      = 1.75,
    eta_decay  = 0.985,
    verbose    = True
)
lpf_svc.fit_importances(x_train.values, y_train)

In [13]:
# Create random array of samples to be used as adversaries
n_classes = lpf_svc.n_classes
targets = np.eye(n_classes)[np.array(
    y_test.apply(lambda x: np.random.choice([i for i in range(n_classes) if i != x]))
)]



(13360, 10)

In [14]:
# # Generate adversaries
# adversaries = lpf_svc.generate(x=x_test.values, y=targets)

LowProFool: 100%|██████████| 15/15 [26:39<00:00, 106.63s/it]


In [15]:
adversaries = np.load("low_pro_fool_adversaries.npy")

In [16]:
x_test.values

array([[-0.55982244, -0.36254078,  0.4757614 , ..., -0.4096874 ,
        -0.7243561 , -0.3420553 ],
       [-0.745914  ,  0.60287666, -0.6751136 , ...,  1.1506999 ,
         0.7386935 , -0.8697235 ],
       [-0.89703625, -0.7198213 , -0.7485737 , ..., -0.62033534,
        -0.8588231 ,  0.12858231],
       ...,
       [-1.0559875 , -0.7046178 , -1.6545817 , ..., -0.6851947 ,
        -0.87508506, -0.9720311 ],
       [ 0.44384784,  0.3216133 , -0.28332636, ..., -0.01097645,
         0.25328344, -0.606425  ],
       [ 0.6305088 ,  1.0817845 ,  0.6961418 , ...,  2.2589543 ,
         1.8368113 ,  0.27051824]], dtype=float32)

In [33]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = svc_clf.predict(adversaries)

accuracy = accuracy_score(y_pred, y_test_flag)
accuracy

0.19281437125748502

In [34]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1875
           1       0.26      0.44      0.32      1465

    accuracy                           0.19      3340
   macro avg       0.13      0.22      0.16      3340
weighted avg       0.11      0.19      0.14      3340



In [17]:
# Check the success rate
expected = np.argmax(targets, axis=1)
predicted = np.argmax(classif_svc.predict(adversaries), axis=1)

correct = (expected == predicted)
success_rate = np.sum(correct) / correct.shape[0]
print("Test set size: {}".format(targets.shape[0]))
print("Success rate:  {:.2f}%".format(100*success_rate))

Test set size: 3340
Success rate:  80.72%


In [26]:
# from art.attacks.evasion import LowProFool

# lpf_svc = LowProFool(
#     classifier = classif_svc,
#     n_steps    = 15,
#     eta        = 15,
#     lambd      = 1.75,
#     eta_decay  = 0.985,
#     verbose    = True
# )
# lpf_svc.fit_importances(x_train.values, y_train)

# Create random array of samples to be used as adversaries
n_classes = lpf_svc.n_classes
targets_train = np.eye(n_classes)[np.array(
    y_train.apply(lambda x: np.random.choice([i for i in range(n_classes) if i != x]))
)]
# Generate adversaries
adversaries_train = lpf_svc.generate(x=x_train.values, y=targets_train)

LowProFool: 100%|██████████| 15/15 [2:35:47<00:00, 623.16s/it] 


In [27]:
np.save('adversaries_train.npy', adversaries_train)

In [28]:
# Initialize and fit SVC
svc_clf_attacked = SVC()
svc_clf_attacked.fit(adversaries_train, y_train) 

SVC()

In [37]:
y_pred_adversarial = svc_clf_attacked.predict(adversaries_train)
print(classification_report(y_train, y_pred_adversarial ))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3390
           1       1.00      1.00      1.00      9970

    accuracy                           1.00     13360
   macro avg       1.00      1.00      1.00     13360
weighted avg       1.00      1.00      1.00     13360



## Testing Adversarially Trained Model on Real Data

In [38]:
y_pred_adversarial_model_test = svc_clf_attacked.predict(x_test)
print(classification_report(y_test, y_pred_adversarial_model_test ))

              precision    recall  f1-score   support

           0       0.12      0.01      0.01       821
           1       0.75      0.98      0.85      2519

    accuracy                           0.74      3340
   macro avg       0.43      0.50      0.43      3340
weighted avg       0.60      0.74      0.65      3340



# Comparing it with Normal SVM Result

In [40]:
y_pred = svc_clf.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90       821
           1       0.96      0.97      0.97      2519

    accuracy                           0.95      3340
   macro avg       0.93      0.93      0.93      3340
weighted avg       0.95      0.95      0.95      3340

